In [1]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image
import torch
from diffusers.utils import make_image_grid, load_image

import os
import requests
import sys
import copy
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, CLIPTextModel,CLIPVisionModelWithProjection,CLIPVisionModel
from diffusers import DDPMScheduler,DDIMScheduler,EulerDiscreteScheduler
from diffusers import AutoencoderKL, UNet2DConditionModel
from diffusers.utils.torch_utils import randn_tensor
from diffusers.utils.peft_utils import set_weights_and_activate_adapters
from peft import LoraConfig
from diffusers.models.attention_processor import (
            AttnProcessor,
            AttnProcessor2_0,
            IPAdapterAttnProcessor,
            IPAdapterAttnProcessor2_0,
        )
import torchvision
from torch.nn import functional as F
from PIL import Image
from typing import List, Optional, Union
def retrieve_timesteps(
    scheduler,
    num_inference_steps: Optional[int] = None,
    device: Optional[Union[str, torch.device]] = None,
    timesteps: Optional[List[int]] = None,
    **kwargs,
):
    """
    Calls the scheduler's `set_timesteps` method and retrieves timesteps from the scheduler after the call. Handles
    custom timesteps. Any kwargs will be supplied to `scheduler.set_timesteps`.

    Args:
        scheduler (`SchedulerMixin`):
            The scheduler to get timesteps from.
        num_inference_steps (`int`):
            The number of diffusion steps used when generating samples with a pre-trained model. If used,
            `timesteps` must be `None`.
        device (`str` or `torch.device`, *optional*):
            The device to which the timesteps should be moved to. If `None`, the timesteps are not moved.
        timesteps (`List[int]`, *optional*):
                Custom timesteps used to support arbitrary spacing between timesteps. If `None`, then the default
                timestep spacing strategy of the scheduler is used. If `timesteps` is passed, `num_inference_steps`
                must be `None`.

    Returns:
        `Tuple[torch.Tensor, int]`: A tuple where the first element is the timestep schedule from the scheduler and the
        second element is the number of inference steps.
    """
    if timesteps is not None:
        accepts_timesteps = "timesteps" in set(inspect.signature(scheduler.set_timesteps).parameters.keys())
        if not accepts_timesteps:
            raise ValueError(
                f"The current scheduler class {scheduler.__class__}'s `set_timesteps` does not support custom"
                f" timestep schedules. Please check whether you are using the correct scheduler."
            )
        scheduler.set_timesteps(timesteps=timesteps, device=device, **kwargs)
        timesteps = scheduler.timesteps
        num_inference_steps = len(timesteps)
    else:
        scheduler.set_timesteps(num_inference_steps, device=device, **kwargs)
        timesteps = scheduler.timesteps
    return timesteps, num_inference_steps

/home/zhouxiaoyu/anaconda3/envs/ose/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# from safetensors import safe_open
# with safe_open("/nvme0/public_data/Occupancy/proj/cache/stabilityai/sd-turbo/sd_turbo.safetensors", framework="pt", device=0) as f:
#     for k in f.keys():
#         print(k)


In [1]:
import torch
from diffusers import FlowMatchEulerDiscreteScheduler
from diffusers import StableDiffusion3Pipeline,StableDiffusion3Img2ImgPipeline
from peft.utils import get_peft_model_state_dict
pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3.5-large-turbo", torch_dtype=torch.bfloat16)
pipe = pipe.to("cuda")



/home/zhouxiaoyu/anaconda3/envs/ose/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...: 100%|██████████| 9/9 [00:02<00:00,  3.22it/s]


In [3]:
pipe.scheduler.sigmas

tensor([1.0000, 0.9819, 0.9624, 0.9414, 0.9187, 0.8940, 0.8672, 0.8379, 0.8057,
        0.7702, 0.7310, 0.6872, 0.6382, 0.5829, 0.5201, 0.4479, 0.3644, 0.2663,
        0.1498, 0.0089, 0.0000], device='cuda:0')

In [2]:
with torch.no_grad():
    image = pipe(
        "Family guy, shut up Meg",
        num_inference_steps=20,
        guidance_scale=0.0,
    ).images[0]

100%|██████████| 20/20 [00:04<00:00,  4.33it/s]


In [ ]:
image

In [ ]:
def get_timesteps(scheduler, num_inference_steps, strength, device):
    # get the original timestep using init_timestep
    init_timestep = min(int(num_inference_steps * strength), num_inference_steps)

    t_start = max(num_inference_steps - init_timestep, 0)
    timesteps = scheduler.timesteps[t_start * scheduler.order :]

    return timesteps, num_inference_steps - t_start

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
torch.cuda.set_device(3)

In [ ]:
# from PIL import Image
# import numpy as np
# image = Image.open("/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/2_car/CAM_FRONT_RIGHT/004.jpg")
# image

In [ ]:
# mask = np.load("/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/2_car_mask/CAM_FRONT_RIGHT/004.npy")
# Image.fromarray(mask.astype(np.uint8)*100)

In [ ]:
# import os
# import cv2
# scene_dir = "/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/static_render_result/2_scene"
# car_dir = "/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/2_car"
# car_mask_dir = "/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/2_car_mask"
# for cam in os.listdir(scene_dir):
#     for f in os.listdir(os.path.join(scene_dir, cam)):
#         if f.endswith(".jpg"):
#             scene_path = os.path.join(scene_dir, cam, f)
#             car_path = os.path.join(car_dir, cam, f)
#             car_mask_path = os.path.join(car_mask_dir, cam, f.replace(".jpg", ".npy"))
#             scene_image = np.array(Image.open(scene_path))
#             car_image =np.array(Image.open(car_path))
#             mask = np.load(car_mask_path)>0
#             #创建一个5*5的值为1的卷积核
#             kernel = np.ones((3,3),np.uint8)
#             #腐蚀运算，iteration=1,迭代腐蚀1次
#             mask = cv2.erode(mask.astype(np.uint8),kernel,iterations=2)>0
#             mask = mask*(car_image!=255).any(-1)
#             mask = mask>0
#             scene_image[mask]=car_image[mask]
#             scene_image = Image.fromarray(scene_image)
#             save_dir = os.path.join(scene_dir.replace("static_render_result","combine"),cam)
#             os.makedirs(save_dir,exist_ok=True)
#             scene_image.save(os.path.join(save_dir,f))

In [ ]:
import numpy as np
x =np.load("/nvme0/public_data/Occupancy/proj/vis/gaussian-splatting/inputs/full/scene0000_00/semantic_mask/0.npy")

In [3]:
2**14*2

32768

In [ ]:
import numpy as np
from PIL import Image
import cv2


def canny_from_pil(image, low_threshold=100, high_threshold=200):
    image = np.array(image)
    image = cv2.Canny(image, low_threshold, high_threshold)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    print(np.unique(image))
    
    control_image = Image.fromarray(image)
    return control_image
image = Image.open("/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/ourscene/gt/00004.png")
image = canny_from_pil(image)

image

In [2]:
vae= AutoencoderKL.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", subfolder="vae").cuda()
vae.eval()

An error occurred while trying to fetch stable-diffusion-v1-5/stable-diffusion-v1-5: Error no file named diffusion_pytorch_model.safetensors found in directory stable-diffusion-v1-5/stable-diffusion-v1-5.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


AutoencoderKL(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0-1): 2 x ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (nonlinearity): SiLU()
          )
        )
        (downsamplers): ModuleList(
          (0): Downsample2D(
            (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
          )
        )
      )
      (1): DownEncoderBlock2D(
        (resnets): ModuleList(
          (0): ResnetBlock2D(
            (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
            (c

In [7]:
len(list(vae.decoder.conv_out.parameters()))

2

In [ ]:
import torchvision.transforms.functional as F
image =load_image("/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/3DRealCar_Renders/image_pairs_output/2024_05_26_19_01_07/gt/00000.png").resize((1024, 1024))
input_image =F.normalize( F.to_tensor(image), mean=[0.5], std=[0.5]).unsqueeze(0).cuda()
# with torch.no_grad():
vae.requires_grad_(False)
with torch.inference_mode():
    latents = vae.encode(input_image).latent_dist.sample() 
    output_image = (vae.decode(latents, return_dict=False, generator=None)[
                        0
                    ]).clamp(-1, 1)
output_image = (output_image + 1) / 2.0
output_image = (output_image * 255).to(torch.uint8)
output_image = output_image.permute(0, 2, 3, 1)[0].cpu().numpy()
output_image = Image.fromarray(output_image)
output_image

In [ ]:
image.min()

In [ ]:
input_image.min()


In [ ]:
image = Image.open("/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/3DRealCar_Renders/image_pairs_output/2024_07_21_17_41_23/renders/00010.png").resize((512, 512))
scheduler =EulerDiscreteScheduler().from_pretrained("stabilityai/sd-turbo", subfolder="scheduler")
num_inference_steps = 5
strength = 1.0
print(f"Num inference steps all: {num_inference_steps}")
print(f"Strength: {strength}")
timesteps, num_inference_steps = retrieve_timesteps(scheduler, num_inference_steps, "cuda", None)
timesteps, num_inference_steps = get_timesteps(scheduler,num_inference_steps, strength, "cuda")
latent_timestep = torch.tensor(timesteps).cuda()
with torch.no_grad():
    input_image = F.normalize( F.to_tensor(image), mean=[0.5], std=[0.5]).unsqueeze(0).cuda()
    latents = vae.encode(input_image).latent_dist.sample() * vae.config.scaling_factor
    noise = randn_tensor(latents.shape, generator=None, device=latents.device, dtype=latents.dtype)
    # latents=scheduler.add_noise(latents, noise, latent_timestep[0:1])
    latents = latents+noise* scheduler.sigmas[3] 
    output_image = (vae.decode(latents / vae.config.scaling_factor, return_dict=False, generator=None)[
                0
            ]).clamp(-1, 1)

In [ ]:
scheduler.sigmas

In [ ]:
latent_timestep

In [ ]:
output_image = (output_image + 1) / 2.0
output_image = (output_image * 255).to(torch.uint8)
output_image = output_image.permute(0, 2, 3, 1)[0].cpu().numpy()
output_image = Image.fromarray(output_image, mode="RGB")
output_image

In [5]:
# prepare image
url = "/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/JiaLei/飞书20250604-153159/001.jpg"
init_image = load_image(url).resize((512, 512))

prompt = "a high quality professional photograph"

In [ ]:
sdturbo_pipe = AutoPipelineForImage2Image.from_pretrained("stabilityai/sd-turbo", torch_dtype=torch.float16, variant="fp16")
sdturbo_pipe.to("cuda")

In [6]:
num_inference_steps = 2
strength = 0.5
image = init_image
for i in range(10):
    image = sdturbo_pipe("a high quality professional photograph",  image=image, num_inference_steps=num_inference_steps, strength=strength, guidance_scale=0).images[0]
    make_image_grid([init_image, image], rows=1, cols=2).save(f"/nvme0/public_data/Occupancy/proj/img2img-turbo/cat_turbo_{i}_{num_inference_steps}_{strength}.png")

NameError: name 'sdturbo_pipe' is not defined

In [10]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

sd21pipeline = AutoPipelineForImage2Image.from_pretrained(
    "/nvme0/public_data/Occupancy/proj/img2img-turbo/stabilityai/sd-turbo", torch_dtype=torch.float16)
sd21pipeline.enable_model_cpu_offload()
# remove following line if xFormers is not installed or you have PyTorch 2.0 or higher installed
sd21pipeline.enable_xformers_memory_efficient_attention()

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  4.21it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [13]:
# pass prompt and image to pipeline
image = init_image
for i in range(1):
    image = sd21pipeline("a high quality professional photograph of a car", image=image, num_inference_steps=2,strength=0.5,guidance_scale=0).images[0]
    make_image_grid([init_image, image], rows=1, cols=2).save(f"/nvme0/public_data/Occupancy/proj/img2img-turbo/outputs/{i}_sd21.png")

100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import make_image_grid, load_image

pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, variant="fp16", use_safetensors=False
)
pipeline.enable_model_cpu_offload()
# remove following line if xFormers is not installed or you have PyTorch 2.0 or higher installed
pipeline.enable_xformers_memory_efficient_attention()

# prepare image
# url = "/nvme0/public_data/Occupancy/proj/img2img-turbo/img2img-init.png"
# init_image = load_image(url)

# prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"

# # pass prompt and image to pipeline
# image = pipeline(prompt, image=init_image,strength=0.5).images[0]
# make_image_grid([init_image, image], rows=1, cols=2)

In [ ]:
image = pipeline("a high quality professional photograph of a car", image=init_image,strength=0.5, guidance_scale=4).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

In [ ]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

image = Image.open("/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/1/CAM_FRONT_RIGHT/001.jpg")

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')



In [ ]:
inputs = processor(images=image, return_tensors="pt")

outputs = model(**inputs)

inputs = processor(images=image, return_tensors="pt")
last_hidden_states = outputs.last_hidden_state

In [ ]:
inputs['pixel_values'].shape

In [3]:
import lpips
import torch
x = torch.rand(1, 3, 224, 224)
loss_fn = lpips.LPIPS(net='vgg')
image1 = x
image2 = x + 0.1 * torch.rand_like(x)
loss = loss_fn(image1, image2)

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/home/zhouxiaoyu/anaconda3/envs/v2v/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zhouxiaoyu/anaconda3/envs/v2v/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/zhouxiaoyu/anaconda3/envs/v2v/lib/python3.11/site-packages/lpips/weights/v0.1/vgg.pth


In [4]:
sd = torch.load("/home/zhouxiaoyu/anaconda3/envs/v2v/lib/python3.11/site-packages/lpips/weights/v0.1/vgg.pth")

In [2]:
import torch
sd = torch.load("/nvme0/public_data/Occupancy/proj/img2img-turbo/outputs/pix2pix_turbo/stage0_render/checkpoints/model_10.pkl")
list(sd["state_dict_tf"].keys())

['patch_embedding.weight',
 'patch_embedding.bias',
 'condition_embedder.time_embedder.linear_1.lora_A.default.weight',
 'condition_embedder.time_embedder.linear_1.lora_B.default.weight',
 'condition_embedder.time_embedder.linear_2.lora_A.default.weight',
 'condition_embedder.time_embedder.linear_2.lora_B.default.weight',
 'condition_embedder.time_proj.lora_A.default.weight',
 'condition_embedder.time_proj.lora_B.default.weight',
 'condition_embedder.text_embedder.linear_1.lora_A.default.weight',
 'condition_embedder.text_embedder.linear_1.lora_B.default.weight',
 'condition_embedder.text_embedder.linear_2.lora_A.default.weight',
 'condition_embedder.text_embedder.linear_2.lora_B.default.weight',
 'blocks.0.attn1.to_q.lora_A.default.weight',
 'blocks.0.attn1.to_q.lora_B.default.weight',
 'blocks.0.attn1.to_k.lora_A.default.weight',
 'blocks.0.attn1.to_k.lora_B.default.weight',
 'blocks.0.attn1.to_v.lora_A.default.weight',
 'blocks.0.attn1.to_v.lora_B.default.weight',
 'blocks.0.attn1.t

In [12]:
from src.loss_utils import ssim,ms_ssim
from torchvision.transforms import functional as F
from PIL import Image
image1 = Image.open("/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/ourscene/gt/00000.png")
image2 = Image.open("/nvme0/public_data/Occupancy/proj/img2img-turbo/inputs/ourscene/renders/00000.png")

In [13]:
image1 = F.to_tensor(image1).unsqueeze(0)
image2 = F.to_tensor(image2).unsqueeze(0)
ssim_value =1- ssim(image1, image2,data_range=1.0, size_average=True)
ms_ssim_value = 1 - ms_ssim(image1, image2, data_range=1.0, size_average=True)

In [14]:
ssim_value, ms_ssim_value

(tensor(0.5890), tensor(0.4981))

In [3]:
77.17+116.06+8.9

202.13000000000002